In [ ]:
import pandas as pd
import quandl

# Define start and end dates
start = pd.to_datetime('2013-01-01')
end = pd.to_datetime('2018-01-01')

# Define ticker symbols in a list
tickers = ['JPM.11', 'C.11', 'BAC.11', 'WFC.11']

# Loop through tickers and download/save data
for ticker in tickers:
  # Download data for current ticker
  data = quandl.get(f'WIKI/{ticker}', start_date=start, end_date=end)
  
  # Extract filename based on ticker symbol
  filename = f"{ticker.split('.')[0]}_CLOSE"  # Remove extension from ticker
  
  # Save data to CSV
  data.to_csv(filename)


# Create a portfolio of stocks
The first step is to import the requisite stocks data we’re going to be working with from quandl. We will work with a bunch of banking stocks namely, JP Morgan, Citi, Bank of America and Wells Fargo. Let’s take 5 years of adjusted close prices from 2013 to 2018 to create a stock portfolio and do our analysis. So, what we have here is five years worth of time series data from 2013 to 2018 in a simple dataframe for each of the four stocks. Each dataframe contains a date column as index and a column for adjusted close price.

# Normalizing Prices
Next, we will normalize the prices by taking the adjusted close price at a particular date and dividing it by the very initial adj. close price. The normalized prices are stored in a new column called ‘Normed Return’.

In [ ]:
for stock_df in (jpm,citi,bofa,wfc):
    stock_df['Normed Return'] = stock_df['Adj.Close']/stock_df.iloc[0]['Adj. Close']
jpm.head()

# Allocations
Next, we’ll assign an array of random weights for the purpose of calculation. These weights will represent the percentage allocation of investments between these four stocks. They must add up to 1.

Let’s assume we had the following allocations for our total portfolio:

30% in JP Morgan Chase
20% in Citi
40% in Bank of America
10% in Wells Fargo
Let’s have these values be reflected by multiplying our Normalized Returns by our allocations.

In [ ]:
for stock_df,allo in zip([jpm,citi,bofa,wfc],[.3,.2,.4,.1]): #Hard coding here
    
    stock_df['Allocation'] = stock_df['Normed Return']*allo
jpm.head()

# Total Portfolio Value
So now we can get a better idea of how the returns are portfolio wise. Let’s assume we invested a million dollars in this portfolio.


In [ ]:
for stock_df in [jpm,citi,bofa,wfc]:
    stock_df['Position Values'] = stock_df['Allocation']*1000000
jpm.head()

Hence, (0.3 x 1 million dollars) is allocated in JP Morgan on day 1. So, this becomes our Position Value for JPM on day 1. At the very next day, since JPM went down the Position Value has now turned to a little less.

We can expand this idea to other stocks and create a larger dataframe containing position values of each stock and also the total portfolio value at the end of each day.

In [ ]:
portfolio_val = pd.concat([jpm['Position Values'],citi['Position Values'],bofa['Position Values'],wfc['Position Values']],axis=1)
portfolio_val.columns = ['JPM Pos','CITI Pos','BOFA Pos','WFC Pos']
portfolio_val['Total Pos'] = portfolio_val.sum(axis=1)

So, now we can see how our position values are changing on a day-to-day basis.

We can also plot the total portfolio values and individual position values on a graph.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
portfolio_val['Total Pos'].plot(figsize=(10,8))
plt.title('Total Portfolio Value')

In [ ]:
portfolio_val.drop('Total Pos',axis=1).plot(kind='line')

In [ ]:
portfolio_val['Daily Return'] = portfolio_val['Total Pos'].pct_change(1)
portfolio_val

In [ ]:
cum_ret = 100 * (portfolio_val['Total Pos'][-1]/portfolio_val['Total Pos'][0] -1 )
print('Our cumulative return is {} percent!'.format(cum_ret))

In [ ]:
portfolio_val['Daily Return'].mean()

In [ ]:
portfolio_val['Daily Return'].std()

In [ ]:
SR = portfolio_val['Daily Return'].mean()/portfolio_val['Daily Return'].std()
SR

In [ ]:
ASR = (252**0.5)*SR
ASR

In [ ]:
# Download and get Daily Returns
jpm = pd.read_csv('JPM_CLOSE',index_col='Date',parse_dates=True)
citi = pd.read_csv('CITI_CLOSE',index_col='Date',parse_dates=True)
bofa = pd.read_csv('BOFA_CLOSE',index_col='Date',parse_dates=True)
wfc = pd.read_csv('WFC_CLOSE',index_col='Date',parse_dates=True)
stocks = pd.concat([jpm,citi,bofa,wfc],axis=1)
stocks.columns = ['jpm','citi','bofa','wfc']
stocks.head()

In [ ]:
log_ret = np.log(stocks/stocks.shift(1))
log_ret.head()

In [ ]:
#calculate the log return mean of each stock
log_ret.mean() * 252

In [ ]:
# Compute pairwise covariance of columns
log_ret.cov()*252

In [ ]:
np.random.seed(101)

# Stock Columns
print('Stocks')
print(stocks.columns)
print('\n')

# Create Random Weights
print('Creating Random Weights')
weights = np.array(np.random.random(4))
print(weights)
print('\n')

# Rebalance Weights
print('Rebalance to sum to 1.0')
weights = weights / np.sum(weights)
print(weights)
print('\n')

# Expected Return
print('Expected Portfolio Return')
exp_ret = np.sum(log_ret.mean() * weights) *252
print(exp_ret)
print('\n')

# Expected Variance
print('Expected Volatility')
exp_vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
print(exp_vol)
print('\n')

In [ ]:
num_ports = 20000

all_weights = np.zeros((num_ports,len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for ind in range(num_ports):

    # Create Random Weights
    weights = np.array(np.random.random(4))

    # Rebalance Weights
    weights = weights / np.sum(weights)
    
    # Save Weights
    all_weights[ind,:] = weights

    # Expected Return
    ret_arr[ind] = np.sum((log_ret.mean() * weights) *252)

    # Expected Variance
    vol_arr[ind] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))

    # Sharpe Ratio
    sharpe_arr[ind] = ret_arr[ind]/vol_arr[ind]

In [ ]:
sharpe_arr.max()

In [ ]:
sharpe_arr.argmax()

In [ ]:
all_weights[1248,:]

In [ ]:
max_sr_ret = ret_arr[1248]
max_sr_vol = vol_arr[1248]
plt.figure(figsize=(17,9))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
# Add red dot for max SR
plt.scatter(max_sr_vol,max_sr_ret,c='red',s=50,edgecolors='black')

In [ ]:
def get_ret_vol_sr(weights):
    """
    Takes in weights and returns back an array of mean return, mean volatility and sharpe ratio
    """
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov() * 252, weights)))
    sr = ret/vol
    return np.array([ret,vol,sr])

In [ ]:
from scipy.optimize import minimize

In [ ]:
def neg_sharpe(weights):
    return  get_ret_vol_sr(weights)[2] * -1

In [ ]:
# Constraints
def check_sum(weights):
    '''
    Returns 0 if sum of weights is 1.0
    '''
    return np.sum(weights) - 1
# By convention of minimize function it should be a function that returns zero for conditions
cons = ({'type':'eq','fun': check_sum})

In [ ]:
# 0-1 bounds for each weight
bounds = ((0, 1), (0, 1), (0, 1), (0, 1))

In [ ]:
# Initial Guess (equal distribution)
init_guess = [0.25,0.25,0.25,0.25]
# Sequential Least Squares Programming (SLSQP).
opt_results = minimize(neg_sharpe,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
opt_results

In [ ]:
opt_results.x

In [ ]:
get_ret_vol_sr(opt_results.x)

In [ ]:
# Our returns go from 0 to somewhere along 0.2
# Create a linspace number of points to calculate x on
frontier_y = np.linspace(0,0.2,100) 
def minimize_volatility(weights):
    return  get_ret_vol_sr(weights)[1]
frontier_volatility = []
for possible_return in frontier_y:
    # function for return
    cons = ({'type':'eq','fun': check_sum},
            {'type':'eq','fun': lambda w: get_ret_vol_sr(w)[0] - possible_return})
    
    result = minimize(minimize_volatility,init_guess,method='SLSQP',bounds=bounds,constraints=cons)
    
    frontier_volatility.append(result['fun'])
plt.figure(figsize=(17,9))
plt.scatter(vol_arr,ret_arr,c=sharpe_arr,cmap='plasma')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
# Add frontier line
plt.plot(frontier_volatility,frontier_y,'g--',linewidth=3)